In [1]:
import numpy as np
import pandas as pd
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate, train_test_split, KFold, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier

uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"

dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290


In [2]:
x = dados[["preco", "idade_do_modelo", "km_por_ano"]]
y = dados["vendido"]

SEED = 158020
np.random.seed(SEED)

treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25, stratify = y)

print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

Treinaremos com 7500 elementos e testaremos com 2500 elementos


In [3]:
dummy_stratified = DummyClassifier()
dummy_stratified.fit(treino_x, treino_y)
acuracia = dummy_stratified.score(teste_x, teste_y) * 100

print("A acurácia do dummy stratified foi %.2f%%" % acuracia)

A acurácia do dummy stratified foi 58.00%


In [4]:
SEED = 158020
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)
previcoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previcoes) * 100
print("A acurácia foi %.2f%%" % acuracia)

A acurácia foi 71.92%


In [5]:
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv=5, return_train_score=False)
results["test_score"]

media = results["test_score"].mean()
desvio_padrao = results["test_score"].std()
print("Accuracy com cross validation, 3 = [%.2f, %.2f]" % ((media - 2 * desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100))

Accuracy com cross validation, 3 = [75.21, 76.35]


# Aleatoriedade no cross validate

In [6]:
def imprime_resultados(results):
  media = results["test_score"].mean()
  desvio = results["test_score"].std()
  print("Accuracy médio: %.2f" % (media * 100))
  print("Intervalo [%.2f, %.2f]" % ((media - 2 * desvio) * 100, (media + 2 * desvio) * 100))

In [7]:
SEED = 301
np.random.seed(SEED)

cross_v = KFold(n_splits = 10, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv=cross_v, return_train_score=False)
imprime_resultados(results)

Accuracy médio: 75.76
Intervalo [73.26, 78.26]


# Aleatoriedade no cross validate com o StratifiedKFold

In [8]:
dados_azar = dados.sort_values("vendido", ascending=True)
x_azar = dados_azar[["preco", "idade_do_modelo", "km_por_ano"]]
y_azar = dados_azar["vendido"]

In [9]:
SEED = 301
np.random.seed(SEED)

cross_v = StratifiedKFold(n_splits = 10, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv=cross_v, return_train_score=False)
imprime_resultados(results)

Accuracy médio: 75.78
Intervalo [73.55, 78.01]


# Criando uma coluna de "modelos" para utilizar o GroupKFold

In [11]:
np.random.seed(SEED)
dados["modelo"] = dados.idade_do_modelo + np.random.randint(-2, 3, size=10000)
dados.modelo = dados.modelo + abs(dados.modelo.min()) + 1
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo
0,30941.02,1,18,35085.22134,18
1,40557.96,1,20,12622.05362,24
2,89627.50,0,12,11440.79806,14
3,95276.14,0,3,43167.32682,6
4,117384.68,1,4,12770.11290,5
